## Read api_key file, and get puuid of a summoner name

In [ ]:
import requests
import time
from methods import get_puuid, get_matches, get_match_info, get_game_patch, get_challengers_solo, get_master_solo, create_connection

with open('D:\\365datascience\\League of Legends Project\\API\\API_key.txt') as f:
    api_key = f.readline()



In [ ]:
summoner_name= 'Rofelzin'

def get_puuid(api_key, summoner_name):
    
    url = f"https://br1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}"
    headers = {
    'X-Riot-Token':api_key
    }
    response = requests.get(url,headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        return data['puuid'], data['id']  # 'id' is the encrypted PUUID
    else:
        print(f"Error fetching data: {response.status_code}")
        return None, None

# Usage
summoner_name= 'Rofelzin'
puuid, encrypted_puuid = get_puuid(api_key, summoner_name)

# Output the results
print(f"PUUID: {puuid}")
print(f"Encrypted PUUID: {encrypted_puuid}")



## Get list of 20 last matches of the summoner

In [ ]:

def get_matches(api_key, puuid):
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    headers = {'X-Riot-Token': api_key}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data: {response.status_code}")
        return None, None

matches = get_matches(api_key, puuid)


In [ ]:
matches

### Connection with the mysql database

In [ ]:


def create_connection():
    return mysql.connector.connect(
        user='root', 
        password='025rafael890',
        host='127.0.0.1',
        database='leagueapi'
    )

def setup_database():
    conn = create_connection()
    cursor = conn.cursor()
    
    cursor.execute('''CREATE TABLE IF NOT EXISTS matches (
                      match_id VARCHAR(255) PRIMARY KEY,
                      champion INT,
                      lane VARCHAR(255))''')
    
    conn.commit()
    conn.close()

setup_database()

## Getting the match info for a specific matchid

In [ ]:
def get_match_info(api_key, matchid):
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchid}"
    headers = {'X-Riot-Token': api_key}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data: {response.status_code}")
        return None, None

match_info = get_match_info(api_key, matches[1])


## Exploring the data

In [ ]:
match_info.keys()

In [ ]:
match_info['info']

In [ ]:
match_info['info']['participants'][0]['placement']

In [ ]:
match_info['info']['participants'][0].keys()

In [ ]:
match_info['info']['participants'][0]['puuid']

In [ ]:
match_info['metadata']['participants'].index('ARoGlv16Prv2cW3pJ4vc4JL9a0YhwpIlU1WP6aB4nXjBc6ydMMJ9Ka4Ia8RRT5txVKDi2wOavGzLJA')

In [ ]:
match_info['info']['teams']

In [ ]:
def get_game_patch(gameVersion):
    patch = float(gameVersion.split('.',2)[0] + '.' + gameVersion.split('.',2)[1])
    return patch

## Method - Insert data in te SQLdatabase

In [ ]:
def store_matches(info):
    conn = create_connection()
    cursor = conn.cursor()
    
    for match in matches:
        match_id = info['gameid']
        champion = info['champion']
        lane = info['lane']
        cursor.execute('INSERT IGNORE INTO matches VALUES (%s, %s, %s)', (match_id, champion, lane))
    
    conn.commit()
    conn.close()


In [ ]:
index = match_info['metadata']['participants'].index(puuid)
gameid = match_info['info']['gameId']
champion =  match_info['info']['participants'][index]['championId']
lane = match_info['info']['participants'][index]['teamPosition']
info = {'gameid':gameid,'champion':champion,'lane':lane}

In [ ]:
info

In [ ]:
store_matches(info)

In [ ]:
matches

## Loop that insert all game data to the database, from the summoner

In [ ]:
for match in matches:
    ## Get match info from API
    match_info = get_match_info(api_key, match)
    
    #Create inputs for DataBase 
    ## CHANGE IT FOR A METHOD TOO
    index = match_info['metadata']['participants'].index(puuid)
    gameid = match_info['info']['gameId']
    champion =  match_info['info']['participants'][index]['championId']
    lane = match_info['info']['participants'][index]['teamPosition']
    info = {'gameid':gameid,'champion':champion,'lane':lane}
    
    #Input into database
    store_matches(info)
    

## Listing Challenger player names


In [ ]:
def get_challengers_solo(api_key):
    url = f"https://br1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5"
    headers = {'X-Riot-Token': api_key}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data: {response.status_code}")
        return None, None

challengers = get_challengers_solo(api_key)

### Exploring Challengers data

In [ ]:
challengers.keys()

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(challengers['entries'])

In [ ]:
df.sort_values('leaguePoints',ascending=False)

## List of all leagues entries - Ranked Solo

In [ ]:
def get_master_solo(api_key, division, tier):
    url = f"https://br1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{division}/{tier}?page=1"
    headers = {'X-Riot-Token': api_key}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data: {response.status_code}")
        return None, None

    
division = 'DIAMOND' # 'IRON', 'BRONZE', 'SILVER','GOLD','PLATINUM','EMERALD','DIAMOND'
tier = 'I' # 'I', 'II', 'III', 'IV'
entries = get_master_solo(api_key, division, tier)

In [ ]:
entries[0]

In [ ]:
df_entries = pd.DataFrame(entrys)

In [ ]:
df_entries.sort_values('leaguePoints',ascending=False)

# With all this info and code, lets start to make a code that will read first 20 players in the soloq Challhenger 5x5 Brazil, and return the game info from this matches.

For this we will need a new DB structure, so we will create a DF with all game info. Then we can choose if we want to drop some, or separate that into different tables in our schema. The process will be: 
- Retrieve information from the top 20 challenger players; (1 req)
- Create a list with all 20 last games from them. This list will contain 400 games ar first; (20 req)
- Drop duplicate gameids, as they may be playing games togheter;
- Retrieve information from this gameids, and store info from all players in the game; (max: 400 req)
- Create a df will all info retrieved;

#### Pre-Project Steps
- Create a .py to contain all relevant functions we have;
- Analyse all info contained into matches and choose if will drop any (https://developer.riotgames.com/apis#match-v5/GET_getMatch)
- Create a mySQL schema, with the choosen info avaible;

## Getting puuid from challengers list

In [ ]:
lista_challenger = df.sort_values('leaguePoints',ascending=False)['summonerName'].tolist()

In [ ]:
lista_challenger[:20]

#### Get a list of matchs from this list

In [ ]:
lista_jogos =[]
count = 0
for player in lista_challenger:
    puuid, encrypted_puuid = get_puuid(api_key, player)
    matches = get_matches(api_key, puuid)
    lista_jogos.append(matches)
    count +=1
    if count == 100:
        time.sleep(120)
        count=0

In [ ]:
lista_jogos_unicos =[] 

In [ ]:
lista_jogos_unicos = [item for sublist in lista_jogos for item in sublist]

In [ ]:
lista_jogos_unicos

In [ ]:
unique_set = set(lista_jogos_unicos)
lista_jogos_unicos = list(unique_set)

In [ ]:
len(lista_jogos_unicos)

### Getting match info from all those matches

In [ ]:
match_info_list = []
count = 0
for match in lista_jogos_unicos:
    match_info = get_match_info(api_key, match)
    if (match_info['info']['gameMode'] == 'CLASSIC' and  match_info['info']['gameType'] == 'MATCHED_GAME'):
        match_info_list.append(match_info)
    count +=1
    if count == 100:
        time.sleep(120)
        count=0

In [ ]:
len(match_info_list)

In [ ]:
match_info_list_keep = match_info_list[397:]

In [ ]:
len(match_info_list_keep)

In [ ]:
match_info_list

#### Exporting as JSON, to use as a model to create SQL DB

In [ ]:
import json
nome_do_arquivo = 'challenger_matchs.json'

In [ ]:
# Abra o arquivo JSON em modo de escrita
with open(nome_do_arquivo, 'w') as arquivo_json:
    # Use a função dump() da biblioteca json para escrever a lista no arquivo JSON
    json.dump(match_info_list_keep, arquivo_json)
